<a href="https://colab.research.google.com/github/luisosmx/API_connections/blob/main/FNI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import http.client
import json

class Client_FNI:
    def __init__(self, key, hash_value):
        self.host = "tstasf.fni-stl.com"
        self.key = key
        self.hash = hash_value

    def post_request(self, endpoint, payload, headers):
        conn = http.client.HTTPSConnection(self.host)
        conn.request("POST", endpoint, payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))

    def create_loan_request(self, applicant1):
        endpoint = "/fni-lenderportal-external/application/1/hi_apply"
        payload = json.dumps({
            "transaction": {
                "key": self.key,
                "hash": self.hash,
                "partnerId": "DATAENTRY"
            },
            "application": {
                "productId": "1",
                "requestCrline": "4200"
            },
            "applicant1": applicant1
        })
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Cookie': 'GCILB="7c7ea07e31036840"'
        }
        response = self.post_request(endpoint, payload, headers)
        return response.get("transaction", {}).get("refnum")

    def check_application_status(self, refnum):
        endpoint = "/fni-lenderportal-external/application/1/hi_status"
        payload = json.dumps({
            "transaction": {
                "key": self.key,
                "hash": self.hash,
                "refnum": refnum
            }
        })
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Cookie': 'GCILB="7c7ea07e31036840"'
        }
        response = self.post_request(endpoint, payload, headers)

        info_status = {
            "refnum": response["transaction"]["refnum"],
            "status": response["transaction"]["status"],
            "application_status": {
                "currStatus": response["application"]["currStatus"],
                "currDecision": response["application"]["currDecision"],
                "timeReceived": response["application"]["timeReceived"],
                "productName": response["field_descriptions"][0]["productName"],
                "productType": response["field_descriptions"][0]["productType"]
            }
        }

        return info_status


# variables
key = ""
hash_value = ""
client = Client_FNI(key, hash_value)

applicant1_data =  {
   "first": "John",
    "mi": "D",
    "last": "Doe",
    "soc": "123456789",
    "dob": "1990-01-01",
    "addr": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "zip": "12345",
    "eMail": "john.doe@example.com",
    "phone": "5564251982",
    "propAddr": "456 Elm St",
    "propCity": "Othertown",
    "propState": "CA",
    "propZip": "54321",
    "addrLengthMos": "12",
    "prevAddr": "789 Oak St",
    "prevAttn": "Apt 1",
    "prevCity": "Anycity",
    "prevState": "CA",
    "prevZip": "54321",
    "gen": "JR",
    "mailAddr": "789 Maple St",
    "mailAttn": "Unit 2",
    "mailCity": "Sometown",
    "mailState": "CA",
    "mailZip": "54321",
    "mmaid": "Doe",
    "altPhone": "9876543210",
    "housingCode": "R",
    "housingCost": "1000"
  }
# request client
refnum = client.create_loan_request(applicant1_data)
print(refnum)
# check status client
status_response = client.check_application_status(refnum)
print(json.dumps(status_response, indent=4))
